<a href="https://colab.research.google.com/github/jaeohshin/deep-learning-with-keras/blob/main/tutorial_8_naturla_languate_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Naturla Language Process (NLP)

In [1]:
!curl -O http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  10.9M      0  0:00:07  0:00:07 --:--:-- 15.8M


In [3]:
!tar -xf aclImdb_v1.tar.gz

In [4]:
!cd aclImdb/
!pwd

/content


In [5]:
!ls

aclImdb  aclImdb_v1.tar.gz  sample_data


In [6]:
!cd aclImdb/

In [7]:
!pwd

/content


In [8]:
!pwd

/content


In [9]:
!rm -r aclImdb/train/unsup

In [10]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [ ]:
# Prepare a validation set by setting aprt 20% of the training text file
#in a new directory, /val

import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        #move files from train dir to val dir
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)
